# Inicialización Grader

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

# Archivos de datos

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 921MB/s]


Se descomprime el archivo con los datos:

In [ ]:
!unzip udea*.zip > /dev/null

# Importaciones

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, classification_report


Se leen los archivos train.csv y test.csv en dataframes de pandas y muestra el tamaño del dataset de entrenamiento (n_filas, n_columnas).

Muestra el tamaño de cada uno (número de filas y columnas) para verificar que se cargaron correctamente.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.shape, test.shape


((692500, 21), (296786, 20))

# Definir variables del problema
Define el nombre de la variable objetivo: RENDIMIENTO_GLOBAL.

Separa:

X: todas las columnas excepto la target. y: la columna objetivo.

Identifica:

numeric_cols: columnas numéricas cat_cols: columnas categóricas

Devuelve ambas listas para ver qué columnas hay en cada grupo.

In [ ]:
target = "RENDIMIENTO_GLOBAL"

X = train.drop(columns=[target])
y = train[target]

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols     = X.select_dtypes(include=['object']).columns.tolist()

numeric_cols, cat_cols


(['ID',
  'PERIODO_ACADEMICO',
  'INDICADOR_1',
  'INDICADOR_2',
  'INDICADOR_3',
  'INDICADOR_4'],
 ['E_PRGM_ACADEMICO',
  'E_PRGM_DEPARTAMENTO',
  'E_VALORMATRICULAUNIVERSIDAD',
  'E_HORASSEMANATRABAJA',
  'F_ESTRATOVIVIENDA',
  'F_TIENEINTERNET',
  'F_EDUCACIONPADRE',
  'F_TIENELAVADORA',
  'F_TIENEAUTOMOVIL',
  'E_PRIVADO_LIBERTAD',
  'E_PAGOMATRICULAPROPIO',
  'F_TIENECOMPUTADOR',
  'F_TIENEINTERNET.1',
  'F_EDUCACIONMADRE'])

# Tomar muestra de 100k para validación cruzada

Toma una muestra estratificada de 100 000 ejemplos del dataset original:

train_size=100_000: número de filas a usar.

stratify=y: mantiene la proporción de las clases.

Solo se usan X_sample y y_sample; el resto descarta.

Muestra las dimensiones de la muestra.

Esto se acelerar la validación cruzada, ya que entrenar una MLP sobre todo el dataset sería mucho más costoso.

In [ ]:
X_sample, _, y_sample, _ = train_test_split(
    X, y,
    train_size=100_000,
    stratify=y,
    random_state=42
)

X_sample.shape, y_sample.shape


((100000, 20), (100000,))

# Preprocesamiento alternativo

Para variables numéricas:

SimpleImputer(strategy="median"): rellena nulos con la mediana.

StandardScaler(): estandariza (media 0, desviación 1).

Para variables categóricas:

SimpleImputer(strategy="most_frequent"): rellena nulos con la categoría más frecuente.

OneHotEncoder(handle_unknown="ignore", sparse_output=False): hace one-hot encoding, ignorando categorías desconocidas y devolviendo un array denso.

Todo se integra en un ColumnTransformer llamado preprocess, que aplica cada pipeline a las columnas correctas.

In [ ]:
numeric_transform = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transform = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer([
    ("num", numeric_transform, numeric_cols),
    ("cat", categorical_transform, cat_cols)
])


# Definicion de parámetros de la Red Neuronal (MLPClassifier)

Arquitectura: dos capas ocultas con 128 y 64 neuronas.

Activación: relu.

Optimizador: adam.

Tasa de aprendizaje inicial: 0.001.

Tamaño de batch: 256.

max_iter=50: máximo de 50 iteraciones de entrenamiento.

early_stopping=False: no usa parada temprana.

random_state=42: reproducibilidad.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    solver="adam",
    learning_rate_init=0.001,
    batch_size=256,
    max_iter=50,
    early_stopping=False,
    random_state=42
)


# Pipeline completo

"prep": aplica el preprocesamiento definido (preprocess: imputación + escalado + one-hot).

"model": aplica la red neuronal mlp.

In [ ]:
pipeline = Pipeline([
    ("prep", preprocess),
    ("model", mlp)
])


Se define un esquema de validación cruzada estratificada con 3 folds.

Ejecuta cross_val_score usando el pipeline (preprocesado + MLP), la muestra X_sample, y_sample y la métrica de accuracy.

Imprime el accuracy medio de la validación cruzada y la desviación estándar de esa métrica.

Así tenemos una idea del rendimiento promedio del modelo sobre los datos no vistos.

In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

scores = cross_val_score(
    pipeline, X_sample, y_sample,
    cv=cv, scoring="accuracy", n_jobs=-1
)

print("Accuracy promedio CV:", scores.mean())
print("Desviación estándar:", scores.std())


Accuracy promedio CV: 0.3772599995754562
Desviación estándar: 0.0027558670258697633


La siguiente celda divide todo el dataset original en 80% entrenamiento (X_train, y_train), 20% validación (X_val, y_val) y estratificado por la variable objetivo.

También se entrena el pipeline en X_train, y_train y calcula e imprime el accuracy en el conjunto hold-out y un classification_report con precisión, recall y F1 por clase.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

pipeline.fit(X_train, y_train)
pred_val = pipeline.predict(X_val)

print("Accuracy hold-out:", accuracy_score(y_val, pred_val))
print(classification_report(y_val, pred_val))


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy hold-out: 0.4227870036101083
              precision    recall  f1-score   support

        alto       0.57      0.58      0.57     35124
        bajo       0.44      0.56      0.49     34597
  medio-alto       0.32      0.28      0.30     34324
  medio-bajo       0.32      0.27      0.29     34455

    accuracy                           0.42    138500
   macro avg       0.41      0.42      0.41    138500
weighted avg       0.41      0.42      0.42    138500



Luego se muestra un diagrama visual de como está construido el pipeline completo. Reentrena el pipeline usando todo el conjunto de entrenamiento completo (X, y).

Esto se hace para aprovechar el 100% de los datos antes de generar las predicciones para Kaggle.

In [ ]:
pipeline.fit(X, y)


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ID', 'PERIODO_ACADEMICO',
                                                   'INDICADOR_1', 'INDICADOR_2',
                                                   'INDICADOR_3',
                                                   'INDICADOR_4']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknow...
                                                   'E_VALORMATRICULAUNIVERSIDAD',
                                                   'E_HORASSEMANATRABAJA',
                                                   'F_ESTRATOVIVIENDA',
                                                   'F_TIENEINTERNET',
                                                   'F_EDUCACIONPADRE',
                                                   'F_TIENELAVADORA',
                                                   'F_TIENEAUTOMOVIL',
                                                   'E_PRIVADO_LIBERTAD',
                                                   'E_PAGOMATRICULAPROPIO',
                                                   'F_TIENECOMPUTADOR',
                                                   'F_TIENEINTERNET.1',
                                                   'F_EDUCACIONMADRE'])])),
                ('model',
                 MLPClassifier(batch_size=256, hidden_layer_sizes=(128, 64),
                               max_iter=50, random_state=42))])

La siguiente celda usa el pipeline entrenado para predecir RENDIMIENTO_GLOBAL sobre el dataframe test, así que crea un dataframe submission con ID tomado de test y RENDIMIENTO_GLOBAL con las predicciones de la MLP.

También Guarda el archivo como submission_mlp.csv.

Muestra las primeras filas de la submission para verificar el formato.

In [ ]:
test_pred = pipeline.predict(test)

submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": test_pred
})

submission.to_csv("submission_mlp.csv", index=False)
submission.head()


,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo
